
## Collaborative Filtering by Matrix Factorization
In collaborative filtering we wish to factorize our ratings matrix into two smaller feature matrices whose product is equal to the original ratings matrix. Specifically, given some partially filled ratings matrix $X\in \mathbb{R}^{m\times n}$, we want to find feature matrices $U \in \mathbb{R}^{m\times k}$ and $V \in \mathbb{R}^{n\times k}$ such that $UV^T = X$. In the case of movie recommendation, each row of $U$ could be features corresponding to a user, and each row of $V$ could be features corresponding to a movie, and so $u_i^Tv_j$ is the predicted rating of user $i$ on movie $j$. This forms the basis of our hypothesis function for collaborative filtering: 

$$h_\theta(i,j) = u_i^T v_j$$

In general, $X$ is only partially filled (and usually quite sparse), so we can indicate the non-presence of a rating with a 0. Notationally, let $S$ be the set of $(i,j)$ such that $X_{i,j} \neq 0$, so $S$ is the set of all pairs for which we have a rating. The loss used for collaborative filtering is squared loss:

$$\ell(h_\theta(i,j),X_{i,j}) = (h_\theta(i,j) - X_{i,j})^2$$

The last ingredient to collaborative filtering is to impose an $l_2$ weight penalty on the parameters, so our total loss is:

$$\sum_{i,j\in S}\ell(h_\theta(i,j),X_{i,j}) + \lambda_u ||U||_2^2 + \lambda_v ||V||_2^2$$

For this assignment, we'll let $\lambda_u = \lambda_v = \lambda$. 

## MovieLens rating dataset
To start off, let's get the MovieLens dataset. This dataset is actually quite large (24+ million ratings), but we will instead use their smaller subset of 100k ratings. We will have to go fetch this from their website. 

* You can download the archive containing their latest dataset release from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip (last updated October 2016). 
* For more details (contents and structure of archive), you can read the README at http://files.grouplens.org/datasets/movielens/ml-latest-README.html
* You can find general information from their website description located at http://grouplens.org/datasets/movielens/. 

For this assignment, we will only be looking at the ratings data specifically. However, it is good to note that there is additional data available (i.e. movie data and user made tags for movies) which could be used to improve the ability of the recommendation system. 

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg as la
import matplotlib
matplotlib.use("svg")
# AUTOLAB_IGNORE_START
%matplotlib inline
# AUTOLAB_IGNORE_STOP
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [3]:
# AUTOLAB_IGNORE_START
movies = pd.read_csv("ml-latest-small/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
ratings.head()
# AUTOLAB_IGNORE_STOP

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## Data preparation

Matrix factorization requires that we have our ratings stored in a matrix of users, so the first task is to take the dataframe and convert it into this format. Note that in general, typically these matrices are extremely large and sparse (especially if we want to process the 24 million ratings), however for the purposes of this assignment a dense representation of this smaller dataset should fit on any machine. 

### Specification
* Split the data by assigning the first $\mathrm{floor}(9n/10)$ permuted entries to be the training set, and the remaining to be the testing set. Use the order given by the permutation. 
* Each row of the ratings matrix corresponds to a user. The first row of the matrix should correspond to the first user (by userID), and so on. This is because the set of user IDs already form a consecutive range of numbers. 
* Each column of the ratings matrix corresponds to a movie. The order of the columns doesn't matter, so long as the resulting list of movie names is accurate. This is because the set of movie IDs does not form a consecutive range of numbers. 
* Each user and movie that exists in the **ratings** dataframe should be present in the ratings matrix, even if it doesn't have any entries. We will only use the movies dataframe to extract the names of the movies. 
* Any entry that does not have a rating should have a default value of 0. 

In [7]:
import math
def process(ratings, movies, P):
    """ Given a dataframe of ratings and a random permutation, split the data into a training 
        and a testing set, in matrix form. 
        
        Args: 
            ratings (dataframe) : dataframe of MovieLens ratings
            movies (dataframe) : dataframe of MovieLens movies
            P (numpy 1D array) : random permutation vector
            
        Returns: 
            (X_tr, X_te, movie_names)  : training and testing splits of the ratings matrix (both 
                                         numpy 2D arrays), and a python list of movie names 
                                         corresponding to the columns of the ratings matrices. 
    """
    unique_users = ratings.userId.unique()
    unique_movies = ratings.movieId.unique()

    movie_index = np.arange(len(unique_movies))
    index_map = {unique_movies[i]: movie_index[i] for i in range(len(unique_movies))}
    
    movies = movies.reset_index()
    movies.set_index('movieId', inplace=True)
    movie_list = [movies.loc[int(x)].title for x in ratings.movieId.unique()]

    partition = math.floor(9*len(P)/10)
    train_index = P[:partition]
    test_index = P[partition: ]

    train_data = ratings.iloc[train_index,:]
    test_data = ratings.iloc[test_index,:]

    X_tr = np.zeros((len(unique_users), len(unique_movies)))
    X_te = np.zeros((len(unique_users), len(unique_movies)))
    
    for xi in train_data.itertuples():
        X_tr[xi.userId-1][index_map[xi.movieId]] = xi.rating
    
    for xi in test_data.itertuples():
        X_te[xi.userId-1][index_map[xi.movieId]] = xi.rating

    return (X_tr, X_te, movie_list)
    

X_tr, X_te, movieNames = process(ratings, movies, np.random.permutation(len(ratings)))
print(X_tr.shape, X_te.shape, movieNames[:5])


(671, 9066) (671, 9066) ['Dangerous Minds (1995)', 'Dumbo (1941)', 'Sleepers (1996)', 'Escape from New York (1981)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)']


## Alternating Minimization for Collaborative Filtering
Now we build the collaborative filtering recommendation system. We will use a method known as alternating least squares. Essentially, we will alternate between optimizing over $U$ and $V$ by holding the other constant. By treating one matrix as a constant, we get exactly a weighted least squares problem, which has a well-known solution. More details can be found in the lecture notes. 

### Specification
* Similar to the softmax regression on MNIST, there is a verbose parameter here that we can use to print our training err and test error. These should decrease (and converge). 
* We can assume a dense representation of all the inputs. 
* We may find it useful to have an indicator matrix W where $W_{ij} = 1$ if there is a rating in $X_{ij}$. 
* We can initialize U,V with random values. 

In [225]:
def error(X, U, V):
    """ Compute the mean error of the observed ratings in X and their estimated values. 
        Args: 
            X (numpy 2D array) : a ratings matrix as specified above
            U (numpy 2D array) : a matrix of features for each user
            V (numpy 2D array) : a matrix of features for each movie
        Returns: 
            (float) : the mean squared error of the observed ratings with their estimated values
        """
    W = (X > 0).astype(np.float64)
    return(np.sum(W * np.square(X - U.dot(V.T))) / np.count_nonzero(W))

    
def train(X, X_te, k, U, V, niters=51, lam=10, verbose=False):
    """ Train a collaborative filtering model. 
        Args: 
            X (numpy 2D array) : the training ratings matrix as specified above
            X_te (numpy 2D array) : the testing ratings matrix as specified above
            k (int) : the number of features use in the CF model
            U (numpy 2D array) : an initial matrix of features for each user
            V (numpy 2D array) : an initial matrix of features for each movie
            niters (int) : number of iterations to run
            lam (float) : regularization parameter
            verbose (boolean) : verbosity flag for printing useful messages
            
        Returns:
            (U,V) : A pair of the resulting learned matrix factorization
    """
    
    reg = lam * np.eye(k)
    W = (X > 0.1)
    u, v = X.shape
    
    for iter_ in range(niters):
        for i in range(u):
            movie_index = W[i,:]
            V_j = V[movie_index,]
            u_update = la.solve(V_j.T.dot(V_j) + reg, V_j.T.dot(X[i, movie_index]))
            U[i,:] = u_update
        
        for j in range(v):
            user_index = W.T[j,:]
            U_i = U[user_index,]
            v_update = la.solve(U_i.T.dot(U_i) + reg, U_i.T.dot(X.T[j, user_index]))
            V[j,:] = v_update
        
        error_ = error(X, U, V)
        test_err = error(X_te, U, V)

        if verbose:
            if iter_ == 0:
                print('Iter |Train Err |Test Err')
            print(iter_,'|',error_, '|', test_err)
        
    return(U,V)
    pass



In [227]:
k = 5
U = 5 * np.random.random((X_tr.shape[0], k))
V = 5 * np.random.random((k, X_tr.shape[1]))
U, V = train(X_tr, X_te, k, np.random.random((X_tr.shape[0], k)), np.random.random((X_tr.shape[1], k)), verbose = True)


Iter |Train Err |Test Err
0 | 0.936135983959 | 13.5778103259
1 | 0.811776377856 | 13.5779735165
2 | 0.742957939643 | 13.578098252
3 | 0.713093423828 | 13.5781388825
4 | 0.698560765099 | 13.5781683375
5 | 0.691472872344 | 13.5781897344
6 | 0.687529626672 | 13.578204259
7 | 0.685054177408 | 13.5782153496
8 | 0.683428017044 | 13.5782257884
9 | 0.682384961828 | 13.5782365508
10 | 0.68173576564 | 13.5782474455
11 | 0.681310935071 | 13.5782578922
12 | 0.680984184181 | 13.5782674141
13 | 0.680688324995 | 13.5782757917
14 | 0.680408386907 | 13.5782830189
15 | 0.680155851912 | 13.5782892087
16 | 0.679940227848 | 13.578294517
17 | 0.679758191477 | 13.578299095
18 | 0.679599727935 | 13.5783030682
19 | 0.679456235691 | 13.5783065316
20 | 0.679323308798 | 13.5783095575
21 | 0.6791998612 | 13.5783122041
22 | 0.679086443626 | 13.5783145225
23 | 0.678984023924 | 13.5783165584
24 | 0.678893381558 | 13.5783183517
25 | 0.678814894817 | 13.5783199349
26 | 0.678748507532 | 13.5783213321
27 | 0.678693766101

In [206]:
def recommend(X, U, V, movieNames):
    """ Recommend a new movie for every user.
        Args: 
            X (numpy 2D array) : the training ratings matrix as specified above
            U (numpy 2D array) : a learned matrix of features for each user
            V (numpy 2D array) : a learned matrix of features for each movie
            movieNames : a list of movie names corresponding to the columns of the ratings matrix
        Returns
            (list) : a list of movie names recommended for each user
    """
    reco = []
    X_hat = U.dot(V.T)
    X_hat[X > 0] = np.float('-Inf')
    for i,x in enumerate(X_hat):
        reco.append(movieNames[np.argmax(x)])

    return(reco)
    
    pass
    
recommendations = recommend(X_tr, U, V, movieNames)
print(recommendations[:10])


['Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', 'Usual Suspects, The (1995)', 'Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', "Schindler's List (1993)", 'Fargo (1996)', 'Godfather, The (1972)', "Schindler's List (1993)"]
